In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os                import walk
from pydub             import AudioSegment
from pydub.utils       import get_array_type
from pydub.utils       import mediainfo
from pydub.silence     import split_on_silence
from datasets          import load_dataset
from torchaudio.utils  import download_asset
from scipy             import signal
from scipy.io          import wavfile
from matplotlib.pyplot import figure
from tqdm              import tqdm
from os                import listdir
from os.path           import isfile, join
from datetime          import datetime
from pyctcdecode       import build_ctcdecoder
from pprint            import pprint
from torch.utils.data  import Dataset, DataLoader

import matplotlib.pyplot as plt
import pandas            as pd
import numpy             as np
import soundfile         as sf

import re
import json
import requests
import whisper
import git
import os
import jiwer
import IPython
import array
import librosa
import torch
import torchaudio

<h1 style="background-color:#28AA9C;"> <center> <a id='start_cell'></a> Table of content: </center></h1>

[Pipeline](#pipeline_cell) </br>
[Police Example](#Police_Example) </br>
[VAD Threshold paramter example](#Threshold_Example) </br>
[Whisper](#Whisper) </br>
[Police Whisper](#police_dataset_whisper_cell) </br>

<h1 style="background-color:#EBA11F;"> <center> Hebrew: word reading rate:  </center></h1>

https://www.globes.co.il/news/article.aspx?did=603261

<p style="color:red;"> 140-150 Words per minute -> seem unreal -> so I took 1.5 words per second </p>


<h1 style="background-color:LightGreen;"> <center> GPU / CPU </center></h1>

In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

<h1 style="background-color:LightGreen;"> <center> <a id='pipeline_cell'></a> Pipeline Utils </center></h1>

[Table Of Content](#start_cell)

In [ ]:
def get_total_speaking_length(speech_timestamps):
    total = 0
    for val in speech_timestamps:
        tmp = val['end'] - val['start']
        total = total + tmp
    return total

In [56]:
def HebrewNormalizer(hebrew_text):
    # --- step 1: remove sign characters
    #
    # ignore_characters = ",~!@#%^&*()-+/|<>[]*'?.{}"
    # for character in ignore_characters:
    #     hebrew_text = hebrew_text.replace(character, '')

        # --- step 2: replace signs
    hebrew_text = hebrew_text.replace('$', " דולר")
    hebrew_text = hebrew_text.replace('₪', " שח")
    hebrew_text = hebrew_text.replace('€', " יורו")
    # hebrew_text = hebrew_text.replace('.', " נקודה")
    hebrew_text = hebrew_text.replace('ת"א', "תל אביב")
    hebrew_text = hebrew_text.replace('ב"ש', "באר שבע")
    hebrew_text = hebrew_text.replace('ע"י', "על ידי")
    hebrew_text = hebrew_text.replace('אח"כ', "אחר כך")
    hebrew_text = hebrew_text.replace('\"', "")

    # for now we will not handle digits, we will have to handle digits if it costs us in the performance of the model
    # TODO: handle dates: 3/7 -> third of july
    # # --- step 3: replace numbers to words
    # dict_nums = {
    #     "0": "אפס",
    #     "1": "אחד",
    #     "2": "שתיים",
    #     "3": "שלוש",
    #     "4": "ארבע",
    #     "5": "חמש",
    #     "6": "שש",
    #     "7": "שבע",
    #     "8": "שמונה",
    #     "9": "תשע",
    #     "10": "עשר",
    # }
    # for digit, word in dict_nums.items():
    #     hebrew_text = hebrew_text.replace(digit, word)
    #
    # # --- step 4: replace female numbers to male numbers
    # dict_male = {
    #     "אחת": "אחד",
    #     "שתיים": "שניים",
    #     "שלושה": "שלוש",
    #     "ארבעה": "ארבע",
    #     "חמישה": "חמש",
    #     "שישה": "שש",
    #     "שבעה": "שבע",
    #     "תשעה": "תשע",
    #
    # }
    # for female, male in dict_male.items():
    #     hebrew_text = hebrew_text.replace(female, male)
    # postproccessing, removing special charcteres after handling and translating them
    valid_tokens = "פ ם ן ו ט א ר ק ף ך ל ח י ע כ ג ד ש ץ ת צ מ נ ה ב ס ז 1 2 3 4 5 6 7 8 9 0"
    valid_tokens = set([x.lower() for x in valid_tokens])
    # The caret in the character class ([^) means match anything but
    invalid_chars_regex = f"[^\s{re.escape(''.join(set(valid_tokens)))}]"


    """ DO ADAPT FOR YOUR USE CASE. this function normalizes the target text. """
    hebrew_text = re.sub(invalid_chars_regex, " ", hebrew_text)
    hebrew_text = re.sub("\s+", " ", hebrew_text).strip()
    # --- return result
    return hebrew_text    
   

In [ ]:
def run_pipeline(vad_model, vad_utils, source_file, dest_file, text, sample_rate, mp3_format=False):
    
    # 1. get silero util functions
    (get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
    
    # 2. get file VAD
    wav_audio         = read_audio(source_file, sampling_rate=sample_rate)       
    full_audio_lenth  = len(wav_audio) / sample_rate
    speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate, threshold=0.4) # see 'bad example' for threshold
    
    # 3. calc staicics
    new_file_length   = get_total_speaking_length(speech_timestamps) / sample_rate
    text_length       = len(text.split())
    if text_length < (1.2*new_file_length):
        file_name = source_file[source_file.rfind("/")+1:]
        print(f"Outlier: {file_name}, Text length: {text_length}, File Length: {full_audio_lenth} seconds, File Without Silence: {new_file_length}, Required Min Words: {(140/60*new_file_length):.2f}")
        return None, None
    
    # if we will used mp3
    if mp3_format:
        # 4. convert to new file (without noise)    
        save_audio("tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
        
        # 5. convert to mp3
        AudioSegment.from_wav("tmp.wav").export(dest_file, format="mp3")
        os.remove("tmp.wav")
    else:
        save_audio(dest_file, collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)

    
    return text_length, new_file_length
    

In [ ]:
def run_on_new_police_data(vad_model, vad_utils, source_folder, dest_folder, sample_rate, gt_excell_path):
    
    # get poilce GT text
    df_gt                  = pd.read_excel(gt_excell_path)
    
    
    # get sub folder list
    folder_list   = [f for f in listdir(source_folder) if not isfile(join(source_folder, f))]        
    df_statistics = pd.DataFrame()

    files_name        = []    
    gt_len            = []
    file_length       = []      
    gt_text           = []
    
    for folder_name in tqdm(folder_list):
        
        # --- get wav file
        splitted_wav_folder = f"{source_folder}/{folder_name}"
        splitted_files      = [entry for entry in os.listdir(splitted_wav_folder) if os.path.isfile(os.path.join(splitted_wav_folder, entry))]
        
        # --- get GT text
        gt_list             = df_gt[df_gt['file'] == f"{folder_name}.mp3"]['text'].values                
                        
        for i in range(len(splitted_files)): 
            part           = "%04d" % (i,)
            full_src       = f"{source_folder}/{folder_name}/{folder_name}_{part}.wav"       
            full_dst       = f"{dest_folder}/{folder_name}_{part}.'wav'"       
            
            
            text_length, new_file_length = run_pipeline(vad_model, 
                                                        vad_utils, 
                                                        full_src, 
                                                        full_dst, 
                                                        HebrewNormalizer(gt_list[i]), 
                                                        sample_rate)
            # if outlier
            if text_length is None:
                continue
            
            gt_text.append(gt_list[i])
            files_name.append(f"{folder_name}_{part}")
            gt_len.append(text_length)
            file_length.append(new_file_length)        
                        

    df_statistics = pd.DataFrame(data = {"File":   files_name,
                                  "Duration":      file_length,
                                  "Text Length":   gt_len,
                                  "Text":          gt_text
                                  })                
    df_statistics.to_csv(f"{dest_folder}/statistics.csv", index=False)
    

<h1 style="background-color:LightGreen;"> <center> <a id='Police_Example'></a>  Example on new police corpus </center></h1>

[Table Of Content](#start_cell)

In [ ]:
vad_model, vad_utils = torch.hub.load(repo_or_dir  = 'snakers4/silero-vad',
                                      model        = 'silero_vad',
                                      force_reload = True,
                                      onnx         = False)


In [ ]:
gt_excell_path = "/home/amitli/Datasets/Unit_Tests/new_corpus.xlsx"
source_folder  = "/home/amitli/Datasets/Unit_Tests/Source"
dest_folder    = "/home/amitli/Datasets/Unit_Tests/Dest"
sample_rate    = 8000 

run_on_new_police_data(vad_model, vad_utils, source_folder, dest_folder, sample_rate, gt_excell_path)

In [ ]:
df = pd.read_csv("/home/amitli/Datasets/Unit_Tests/Dest/statistics.csv")
df.head()

<h1 style="background-color:#AA2849;"> <center> <a id='Threshold_Example'></a> Example why I used threshold=0.4 (instead of 0.5) </center></h1>

[Table Of Content](#start_cell)

<p style="color:green;">GT Text: "אולגה ובעלה יהושע אז בעצם מהי"</p>

In [ ]:
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav")

<p style="color:red;">Result Text:  missing "יהושע"</p>

In [ ]:
(get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
sample_rate       = 8000
wav_audio         = read_audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav", sampling_rate=sample_rate)       
speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate)        
save_audio("/home/amitli/Datasets/Unit_Tests/tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/tmp.wav")

<p style="color:blue;">Try to manual fix:</p>

In [ ]:
(get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
sample_rate       = 8000
wav_audio         = read_audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav", sampling_rate=sample_rate)       
speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate, threshold=0.4)        
save_audio("/home/amitli/Datasets/Unit_Tests/tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/tmp.wav")

<h1 style="background-color:LightGreen;"> <center> <a id='Whisper'></a> Whisper </center></h1>

[Table Of Content](#start_cell)

In [ ]:
def Get_Whisper_Text(whisper_model, file_name, device):
    
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(file_name)
    audio = whisper.pad_or_trim(audio)
    mel   = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

    # decode the audio
    options = whisper.DecodingOptions(language = 'he', beam_size=8, patience=2, fp16 = False)    #, temperature=0.2
    result  = whisper.decode(whisper_model, mel, options)
    result  = result.text
    result  = HebrewNormalizer(result)
    
    return result

In [ ]:
whisper_model = whisper.load_model("large", device="cpu")

<p style="color:#28AA9C; font-size:24px;"> Run on first 2 rows from dataframe and add Whisper results: </p>

In [ ]:
df             = pd.read_csv("/home/amitli/Datasets/Unit_Tests/Dest/statistics.csv")
whisper_column = [None]*df.shape[0]
whisper_wer    = [None]*df.shape[0]

for i in range(2):
    audio_file        = f"{dest_folder}/{df['File'].values[i]}.mp3"
    whisper_text      = Get_Whisper_Text(whisper_model, audio_file, "cpu")
    whisper_column[i] = (whisper_text)
    whisper_wer[i]    = jiwer.wer(df["Text"].values[i], whisper_text)
    
    
df["Whisper"] = whisper_column
df["WER"]     = whisper_wer

df.to_csv(f"{dest_folder}/whisper.csv", index=False)

In [ ]:
df = pd.read_csv(f"{dest_folder}/whisper.csv")
df.head()

In [ ]:
def Get_Whisper_Transcribe(whisper_model, file_name):            
    result = whisper_model.transcribe(file_name)
    return result['text']

<h1 style="background-color:#AA2849;"> <center> <a id='Threshold_Example'></a> How can it be ??? </center></h1>

In [ ]:
police_source_file = "/home/amitli/Datasets/ASR_Police/new_corpus/-37_ZAraHQM.mp3"
print(f"Police File SR: {mediainfo(police_source_file)['sample_rate']}")

In [ ]:
test_full_file = "/home/amitli/Datasets/Unit_Tests/Source/-3dd58g7rQQ/-3dd58g7rQQ_0001.wav"
mp3_file       = "/home/amitli/Datasets/test_mp3.mp3"
wav_file       = "/home/amitli/Datasets/test_wav.wav"

# 1. get source sample rate:
print(f"Source file SR: {mediainfo(test_full_file)['sample_rate']}")
    

# 2. save as mp3
run_pipeline(vad_model, vad_utils, test_full_file, mp3_file, "a b c d e f", sample_rate=8000, mp3_format=True)
print(f"MP3 file SR: {AudioSegment.from_mp3(mp3_file).frame_rate}")

# 3. save as wav
run_pipeline(vad_model, vad_utils, test_full_file, wav_file, "a b c d e f", sample_rate=8000, mp3_format=False)
print(f"WAV file SR: {mediainfo(wav_file)['sample_rate']}")

mp3_text = Get_Whisper_Text(whisper_model, mp3_file, "cpu")
wav_text = Get_Whisper_Text(whisper_model, wav_file, "cpu")

mp3_wer  = jiwer.wer(df["Text"].values[1], mp3_text)
wav_wer  = jiwer.wer(df["Text"].values[1], wav_text)

print(f"MP3 WER: {mp3_wer:.3f}")
print(f"WAV WER: {wav_wer:.3f}")

In [ ]:
print(f"GT : {df['Text'].values[1]}")
print(f"MP3: {mp3_text}")
print(f"WAV: {wav_text}")

In [ ]:
normalize_mp3 = HebrewNormalizer(mp3_text)
normalize_wav = HebrewNormalizer(wav_text)


print(f"GT : {df['Text'].values[1]}")
print(f"Normalized MP3: {normalize_mp3}")
print(f"Normalized WAV: {normalize_wav}")

mp3_wer  = jiwer.wer(df["Text"].values[1], normalize_mp3)
wav_wer  = jiwer.wer(df["Text"].values[1], normalize_wav)

print(f"MP3 WER: {mp3_wer:.3f}")
print(f"WAV WER: {wav_wer:.3f}")

<h1 style="background-color:#28AA9C;"> <center> <a id='police_dataset_whisper_cell'></a> Run Police Whisper </center></h1>

[Table Of Content](#start_cell)

In [57]:
df_gt         = pd.read_excel("/home/amitli/Datasets/MyVoiceTests_16SR/new_corpus.xlsx")
source_folder = "/home/amitli/Datasets/MyVoiceTests_16SR"

In [59]:
class PoliceDataset(Dataset):
        
    def __init__(self, source_folder, df_gt, device, filter_files=None):        
        
        self.folder_list   = [f for f in listdir(source_folder) if not isfile(join(source_folder, f))]        
        self.l_main_file   = []
        self.l_sub_file    = []
        self.l_full_path   = []
        self.l_gt          = []
        self.device        = device

        for folder_name in tqdm(self.folder_list):
                        
            # --- get wav file
            splitted_wav_folder = f"{source_folder}/{folder_name}"
            splitted_files      = [entry for entry in os.listdir(splitted_wav_folder) if os.path.isfile(os.path.join(splitted_wav_folder, entry))]

            # --- get GT text
            gt_list             = df_gt[df_gt['file'] == f"{folder_name}.mp3"]['text'].values                

            for i in range(len(splitted_files)): 
                part           = "%04d" % (i,)
                full_src       = f"{source_folder}/{folder_name}/{folder_name}_{part}.wav"       
                
                if filter_files is not None:                      
                    if full_src not in filter_files:
                        continue

                self.l_main_file.append(folder_name)
                self.l_sub_file.append(f"{folder_name}_{part}.wav")
                self.l_full_path.append(full_src)                
                self.l_gt.append(gt_list[i])
        

    def __len__(self):
        return len(self.l_gt)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        audio_file_path = self.l_full_path[idx]        
        audio           = whisper.load_audio(str(audio_file_path))
        audio           = whisper.pad_or_trim(audio)
        
        mel             = whisper.log_mel_spectrogram(audio).to(self.device)        
        
        sample          = {'mel':      mel, 
                          'text':      HebrewNormalizer(self.l_gt[idx]), 
                          'main_file': self.l_main_file[idx],
                          'sub_file':  self.l_sub_file[idx],
                          'full_path': audio_file_path}
        
        return sample

In [60]:
dataset = PoliceDataset(source_folder = "/home/amitli/Datasets/MyVoiceTests_16SR",
                               df_gt  = df_gt,
                               device = DEVICE)

loader = DataLoader(dataset, batch_size=20, shuffle=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 574.29it/s]


In [39]:
len(dataset)

23596

In [ ]:
res

In [71]:
def run_whisper_on_rambo(loader, res_file_name, lang):
    df = pd.DataFrame()
    for batch in tqdm(loader):
        
        languages        = []
        if lang is not None:
            languages = [lang]        
        mel              = batch['mel']
        audio_data       = {'wav': json.dumps(mel.tolist()), 'languages': languages}
        gt               = batch['text']

        #res              = requests.get('http://10.53.140.33:80/batch_inference/', json=audio_data)
        res              = requests.get('http://10.53.140.33:80/batch_inference_beam/', json=audio_data)
        res_list         = res.json()[0]

        l_wer            = []
        l_whisper        = []    
        l_res_lang       = []
        l_avg_logprob    = []
        l_no_speech_prob = []
        l_compres_ratio  = []

        for i, res in enumerate(res_list):
            whisper_text = res['text']
            whisper_text = HebrewNormalizer(whisper_text)
            l_whisper.append(whisper_text)                
            if whisper_text == '':
                l_wer.append(1)
            else:
                l_wer.append(jiwer.wer(whisper_text, gt[i]))
            l_res_lang.append(res['language'])
            l_avg_logprob.append(res['avg_logprob'])
            l_no_speech_prob.append(res['no_speech_prob'])
            l_compres_ratio.append(res['compression_ratio'])


        df_tmp     = pd.DataFrame({
            "whisper":           l_whisper,
            "gt":                gt,
            "wer":               l_wer,
            "main_file":         batch['main_file'],
            "sub_file":          batch['sub_file'],
            "full_path":         batch['full_path'],
            "detect_lang":       l_res_lang,
            "avg_logprob":       l_avg_logprob,
            "no_speech_prob":    l_no_speech_prob,
            "compression_ratio": l_compres_ratio,

        })
        df = pd.concat([df, df_tmp], ignore_index=True)
        #df.to_csv("/home/amitli/Datasets/whisper_beam_10.csv")
        df.to_csv(res_file_name)
        
            
                        

In [ ]:
df.head()

In [4]:
df_naive   = pd.read_csv("/home/amitli/Datasets/whisper.csv")
df_beam_5  = pd.read_csv("/home/amitli/Datasets/whisper_beam_5.csv")
df_beam_10 = pd.read_csv("/home/amitli/Datasets/whisper_beam_10.csv")

In [5]:
len(df_naive), len(df_beam_5), len(df_beam_10)

(23596, 23596, 23596)

In [6]:
naive_hebrew   = round(np.sum(df_naive['detect_lang'].values == 'he')/len(df_naive), 2)
beam_5_hebrew  = round(np.sum(df_beam_5['detect_lang'].values == 'he')/len(df_beam_5), 2)
beam_10_hebrew = round(np.sum(df_beam_10['detect_lang'].values == 'he')/len(df_beam_10), 2)
naive_hebrew, beam_5_hebrew, beam_10_hebrew

(0.92, 0.92, 0.92)

In [7]:
naive_wer    = round(np.mean(df_naive[(df_naive['detect_lang'].values == 'he')]['wer'].values), 2)
beam_5_wer   = round(np.mean(df_beam_5[(df_beam_5['detect_lang'].values == 'he')]['wer'].values), 2)
beam_10_wer  = round(np.mean(df_beam_10[(df_beam_10['detect_lang'].values == 'he')]['wer'].values), 2)

naive_wer, beam_5_wer, beam_10_wer

(0.54, 0.47, 0.46)

In [8]:
naive_under_30    = round(np.sum(df_naive['wer'].values < 0.3)/len(df_naive), 2)
beam_5_under_30   = round(np.sum(df_beam_5['wer'].values < 0.3)/len(df_beam_5), 2)
beam_10_under_30  = round(np.sum(df_beam_10['wer'].values < 0.3)/len(df_beam_10), 2)

c_naive_under_30    = int(np.sum(df_naive['wer'].values < 0.3))
c_beam_5_under_30   = int(np.sum(df_beam_5['wer'].values < 0.3))
c_beam_10_under_30  = int(np.sum(df_beam_10['wer'].values < 0.3))

naive_under_30, beam_5_under_30, beam_10_under_30, c_naive_under_30, c_beam_5_under_30, c_beam_10_under_30

(0.27, 0.31, 0.32, 6461, 7377, 7548)

In [9]:
naive_under_5    = round(np.sum(df_naive['wer'].values < 0.05)/len(df_naive), 2)
beam_5_under_5   = round(np.sum(df_beam_5['wer'].values < 0.05)/len(df_beam_5), 2)
beam_10_under_5  = round(np.sum(df_beam_10['wer'].values < 0.05)/len(df_beam_10), 2)

naive_under_5, beam_5_under_5, beam_10_under_5

(0.03, 0.04, 0.04)

In [10]:
df_results          = pd.DataFrame()
df_results["Type"]     = ["Hebrew", "WER", "WER under 30%", "Total under 30%", "WER under 5%"]
df_results["Naive"]    = [naive_hebrew, naive_wer, naive_under_30, c_naive_under_30, naive_under_5]
df_results["Beam_5"]   = [beam_5_hebrew, beam_5_wer, beam_5_under_30, c_beam_5_under_30, beam_5_under_5]
df_results["Beam_10"]  = [beam_10_hebrew, beam_10_wer, beam_10_under_30, c_beam_10_under_30, beam_10_under_5]
df_results.head()

,Type,Naive,Beam_5,Beam_10
0,Hebrew,0.92,0.92,0.92
1,WER,0.54,0.47,0.46
2,WER under 30%,0.27,0.31,0.32
3,Total under 30%,6461.00,7377.00,7548.00
4,WER under 5%,0.03,0.04,0.04


<u> <p style="color:blue; font-size:24px"> Hebrew Tests:</p> </u>

In [73]:
run_whisper_on_none_heb = False

if run_whisper_on_none_heb is True:
    test_not_heb   = df_beam_10[df_beam_10['detect_lang'] != 'he']['full_path'].values[:20]
    dataset_no_heb = PoliceDataset(source_folder = "/home/amitli/Datasets/MyVoiceTests_16SR",
                                   df_gt         = df_gt,
                                   device        = DEVICE, 
                                   filter_files  = test_not_heb)
    loader_no_heb  = DataLoader(dataset_no_heb, batch_size=20, shuffle=False)

    run_whisper_on_rambo(loader_no_heb, "/home/amitli/Datasets/whisper_beam_10_heb.csv", "he")    

In [78]:
df_no_heb = pd.read_csv("/home/amitli/Datasets/whisper_beam_10_heb.csv")
df_no_heb['whisper'].values

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan])

<u> <p style="color:blue; font-size:24px"> First & Last word </p> </u>

In [11]:
first_equal = 0
last_equal  = 0
total       = 0

for i in range(len(df_beam_10)):
    
    if df_beam_10['detect_lang'].values[i]  != "he":
        continue
        
    whisper = str(df_beam_10["whisper"].values[i])
    if len(whisper.split()) <= 1:
        continue
        
    total         = total + 1
    
    first_gt_word = df_beam_10['gt'].values[i].split(" ")[0]
    last_gt_word  = df_beam_10['gt'].values[i].split(" ")[-1]
               
    first_whisper_word = df_beam_10['whisper'].values[i].split(" ")[0]
    last_whisper_word  = df_beam_10['whisper'].values[i].split(" ")[-1]
               
    if first_gt_word == first_whisper_word:
        first_equal = first_equal + 1
               
    if last_gt_word == last_whisper_word:
        last_equal = last_equal + 1

first_equal = round(first_equal/total, 2)
last_equal  = round(last_equal/total, 2)

first_equal, last_equal

(0.38, 0.32)